In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from datetime import date, timedelta
from pyspark.sql import Row


def get_dates_df():
    dummy_df = spark.createDataFrame([Row(dummy='x')])
    in_dates_df = dummy_df.select(F.explode(F.sequence(F.lit("2020-01-01").cast(T.DateType()), F.lit("2020-12-31").cast(T.DateType()))).alias("flight_date"))

    return in_dates_df


spark = SparkSession.builder.master("local[*]").appName('ex3_add_dates').getOrCreate()

flights_df = spark.read.parquet('s3a://spark/data/stg/flight_matched/')
dates_df = get_dates_df()

dates_full_df = dates_df \
    .withColumn('day_of_week', F.dayofweek(F.col('flight_date'))) \
    .withColumn('day_of_month', F.dayofmonth(F.col('flight_date')))

max_date_df = dates_full_df \
    .groupBy(F.col('day_of_week'), F.col('day_of_month')) \
    .agg(F.max(F.col('flight_date')).alias('flight_date'))

enriched_flights_df = flights_df.join(max_date_df, ['day_of_week', 'day_of_month'])

enriched_flights_df.write.parquet('s3a://spark/data/transformed/flights/',mode='overwrite')

spark.stop()


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from datetime import date, timedelta
from pyspark.sql import Row

In [23]:
spark = SparkSession.builder.master("local[*]").appName('ex3_add_dates').getOrCreate()

def get_dates_df():
    dummy_df = spark.createDataFrame([Row(dummy='x')])
    # dummy_df.show()
    in_dates_df = dummy_df.select(F.explode(F.sequence(F.lit("2020-01-01").cast(T.DateType()), F.lit("2020-12-31").cast(T.DateType()))).alias("flight_date"))
    # in_dates_df.show()

    return in_dates_df

In [21]:
dummy_df1 = get_dates_df()

+-----+
|dummy|
+-----+
|    x|
+-----+

+-----------+
|flight_date|
+-----------+
| 2020-01-01|
| 2020-01-02|
| 2020-01-03|
| 2020-01-04|
| 2020-01-05|
| 2020-01-06|
| 2020-01-07|
| 2020-01-08|
| 2020-01-09|
| 2020-01-10|
| 2020-01-11|
| 2020-01-12|
| 2020-01-13|
| 2020-01-14|
| 2020-01-15|
| 2020-01-16|
| 2020-01-17|
| 2020-01-18|
| 2020-01-19|
| 2020-01-20|
+-----------+
only showing top 20 rows



In [13]:
s = F.sequence(F.lit("2020-01-01").cast(T.DateType()), F.lit("2020-12-31").cast(T.DateType()))

In [16]:
s

Column<'sequence(CAST(2020-01-01 AS DATE), CAST(2020-12-31 AS DATE))'>

In [19]:
# Step 1: Generate the sequence of dates
s = F.sequence(F.lit("2020-01-01").cast(T.DateType()), F.lit("2020-12-31").cast(T.DateType()))

# Step 2: Create a dummy DataFrame with a single row
dummy_df = spark.createDataFrame([Row(dummy='x')])
dummy_df.show()



+-----+
|dummy|
+-----+
|    x|
+-----+



In [20]:
# Step 3: Add the sequence as a column in the DataFrame
df_with_sequence = dummy_df.select(s.alias("date_sequence"))

df_with_sequence.show()



+--------------------+
|       date_sequence|
+--------------------+
|[2020-01-01, 2020...|
+--------------------+



In [ ]:
# Step 4: Explode the date sequence to turn it into individual rows
exploded_dates_df = df_with_sequence.select(F.explode("date_sequence").alias("date"))

# Step 5: Show the result (this will print the sequence of dates)
exploded_dates_df.show(truncate=False)

In [27]:
flights_df = spark.read.parquet('s3a://spark/data/stg/flight_matched/')
flights_df.show()
dates_df = get_dates_df()
dates_df.show()



+------------+-----------+-------+-----------------+---------------+---------+---------+
|day_of_month|day_of_week|carrier|origin_airport_id|dest_airport_id|dep_delay|arr_delay|
+------------+-----------+-------+-----------------+---------------+---------+---------+
|          18|          4|     DL|            11278|          14869|        1|        3|
|          19|          5|     DL|            11433|          14869|        1|       -2|
|          19|          5|     DL|            11433|          10721|       -3|      -12|
|          19|          5|     DL|            14730|          10397|       85|       95|
|          16|          2|     DL|            12889|          10397|       -2|      -17|
|          17|          3|     DL|            11433|          11298|       -5|       -3|
|          18|          4|     DL|            12478|          14107|       -1|      -18|
|          18|          4|     DL|            11618|          10397|       -2|       -6|
|          18|       

In [28]:
dates_full_df = dates_df \
    .withColumn('day_of_week', F.dayofweek(F.col('flight_date'))) \
    .withColumn('day_of_month', F.dayofmonth(F.col('flight_date')))

dates_full_df.show()

+-----------+-----------+------------+
|flight_date|day_of_week|day_of_month|
+-----------+-----------+------------+
| 2020-01-01|          4|           1|
| 2020-01-02|          5|           2|
| 2020-01-03|          6|           3|
| 2020-01-04|          7|           4|
| 2020-01-05|          1|           5|
| 2020-01-06|          2|           6|
| 2020-01-07|          3|           7|
| 2020-01-08|          4|           8|
| 2020-01-09|          5|           9|
| 2020-01-10|          6|          10|
| 2020-01-11|          7|          11|
| 2020-01-12|          1|          12|
| 2020-01-13|          2|          13|
| 2020-01-14|          3|          14|
| 2020-01-15|          4|          15|
| 2020-01-16|          5|          16|
| 2020-01-17|          6|          17|
| 2020-01-18|          7|          18|
| 2020-01-19|          1|          19|
| 2020-01-20|          2|          20|
+-----------+-----------+------------+
only showing top 20 rows



In [29]:
max_date_df = dates_full_df \
    .groupBy(F.col('day_of_week'), F.col('day_of_month')) \
    .agg(F.max(F.col('flight_date')).alias('flight_date'))

max_date_df.show()

enriched_flights_df = flights_df.join(max_date_df, ['day_of_week', 'day_of_month'])

enriched_flights_df.write.parquet('s3a://spark/data/transformed/flights/',mode='overwrite')

spark.stop()

+-----------+------------+-----------+
|day_of_week|day_of_month|flight_date|
+-----------+------------+-----------+
|          3|          22| 2020-12-22|
|          3|          30| 2020-06-30|
|          3|          15| 2020-12-15|
|          7|          21| 2020-11-21|
|          5|          16| 2020-07-16|
|          6|          20| 2020-11-20|
|          4|          10| 2020-06-10|
|          6|           1| 2020-05-01|
|          3|           1| 2020-12-01|
|          7|           4| 2020-07-04|
|          2|           2| 2020-11-02|
|          6|          22| 2020-05-22|
|          1|          25| 2020-10-25|
|          6|          25| 2020-12-25|
|          2|          26| 2020-10-26|
|          3|          13| 2020-10-13|
|          7|          31| 2020-10-31|
|          7|          14| 2020-11-14|
|          2|          19| 2020-10-19|
|          7|          25| 2020-07-25|
+-----------+------------+-----------+
only showing top 20 rows

